In [ ]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
from sklearn.metrics import roc_curve, auc
import scipy
warnings.filterwarnings('ignore')

data_version = "TMD_cov"
output_version = "20240910"

outdir = "/media/hieunguyen/HNSD_mini/outdir"
PROJECT = "TMD450_TCGA_data_analysis"
thres_hypo = 0.3
thres_hyper = 0.6
mode = "all"

summarydf = pd.DataFrame()
selected_candi_features = "ratio_raw"
input_cancer_class = "PAN_CANCER"

path_to_tf_output = os.path.join(outdir, PROJECT, output_version, "TF_output")
path_to_02_tf_output = os.path.join(path_to_tf_output, "02_output")

path_to_savefile = os.path.join(path_to_02_tf_output, "output", input_cancer_class, selected_candi_features)
os.system(f"mkdir -p {path_to_savefile}")

metadata = pd.read_excel("metadata_cfDNA_lowpdepth_TMD_bam_cov.xlsx")
metadata = metadata[metadata["Label"].isin([input_cancer_class, "Control"])]

candidf = pd.read_excel(os.path.join(path_to_08_output, "candi_reads_all_{}_vs_control.xlsx".format(input_cancer_class)))
ichorcnadf = pd.read_excel("./ichorCNA/metadata_cfDNA_lowpdepth.xlsx")

# keep only samples that have ichorCNA tumor fractions. 
ichorcnadf = ichorcnadf[(ichorcnadf["SampleID"].isin(candidf["SampleID"].unique())) & 
                        (ichorcnadf["ichorCNA"].isna() == False)][["SampleID", "ichorCNA"]]

tf_sampledf = pd.read_csv("./ichorCNA/analytical_LOD_TF.csv")
tf_sampledf['Label'] = tf_sampledf["LABEL"].apply(lambda x: "CRC" if x == "Colorectal cancer" else x.split(" ")[0])
tf_sampledf["SampleID"] = tf_sampledf["Sample"].values
tf_sampledf = tf_sampledf[tf_sampledf["Label"].isin(["Control", input_cancer_class])]
tf_candi_sampledf = pd.read_excel(os.path.join(path_to_12_output, "candi_reads_all_{}_vs_control.xlsx".format(input_cancer_class)))

tf_maindf = tf_sampledf[["SampleID", "Label", "Actual tumor_fraction_ichorCNA", "Simulated TF",	"spike-in"]].merge(tf_candi_sampledf[["SampleID", selected_candi_features]], right_on = "SampleID", left_on = "SampleID")
tf_maindf.columns = ["SampleID", "Label", "ichorCNA", "simulated_TF", "spike_in", selected_candi_features]
tf_maindf["spike_in_label"] = tf_maindf["simulated_TF"].apply(lambda x: "Control" if x == "Healthy-control" else input_cancer_class)
convert_ratio = {
    '50' : 0.5, 
    '100': 1, 
    '0.5': 0.005, 
    '25': 0.25, 
    '15': 0.15, 
    '5': 0.5, 
    '1': 1, 
    'HC': 0
}

tf_maindf["spike_in_ratio"] = tf_maindf["spike_in"].apply(lambda x: convert_ratio[x])
maindf = candidf[["SampleID", "Label", selected_candi_features]].merge(ichorcnadf, right_on = "SampleID", left_on = "SampleID")